# Dashboard Model

In [1]:
import pickle 
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split

### Load and Split Data

In [2]:
data = pd.read_csv('HADAR - encoded.csv')

In [3]:
features = data.drop(columns='SalePrice')
target = data.SalePrice

In [4]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=.2, random_state=0)

In [5]:
X_train.shape

(2064, 202)

### Feature Selection

In [6]:
clv = Lasso(random_state=0).fit(X_train, y_train)
# search = GridSearchCV(Lasso(random_state=0, tol=.1), 
#                       {'alpha': np.arange(0.01,1000,1)},
#                      cv=5,
#         scoring='neg_mean_squared_error'
#                      )
# search.fit(X_train, y_train)
# search.best_params_

# pickle.dump(search, open('flask_fsgs.sav', 'wb'))
search = pickle.load(open('flask_fsgs.sav', 'rb'))
search.best_params_

coefficients = search.best_estimator_.coef_
importance = np.abs(coefficients)
features = X_train.columns
keep_feat = np.array(features)[importance > 0]
X_train = X_train[keep_feat]
X_test = X_test[keep_feat]

df_imp = pd.DataFrame(zip(keep_feat,importance)).sort_values(1, ascending = False)
df_imp = df_imp.loc[df_imp[1]>0]
df_imp[:20]

C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 374943093793.1978, tolerance: 1125382511.83553
  model = cd_fast.enet_coordinate_descent(


,0,1
3,OverallQual,14275.988180
19,GarageYrBlt,5291.811439
4,OverallCond,5284.515839
22,WoodDeckSF,3964.934838
40,GarageQual_Enc,3896.934882
51,Neighborhood_NoRidge,3489.962449
64,Exterior1st_Wd Sdng,3459.018386
24,EnclosedPorch,3219.628805
50,Neighborhood_NWAmes,3027.247657
45,Neighborhood_BrkSide,2952.519320


In [7]:
X = X_train[keep_feat]

In [8]:
las = Lasso(random_state=0)
las.fit(X,y_train)

C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 473037590191.9823, tolerance: 1125382511.83553
  model = cd_fast.enet_coordinate_descent(


Lasso(random_state=0)

In [9]:
check_pred = las.predict(X_test[keep_feat])
r2_score(y_test, check_pred)

0.9114043712008295

In [10]:
# save 10 feature set
dummies = ['Neighborhood',
           'BldgType', 
           'HouseStyle',
           'Exterior1st',
          'Exterior2nd',
          'MasVnrType',
          'Foundation',
          'Electrical',
          'GarageType',
          'SaleType',
          'SaleCondition']
# cols = [c for c in keep_feat if c[] != 'test']
# df.loc[:,~df.columns.str.contains('^test', case=False)] 

In [11]:
strings = ["one", "two", "three"]
substring = "wo"

strings_with_substring = [string for string in strings if substring in string]
strings_with_substring

['two']

In [12]:
######### EXCLUDE DUMMIFIED VARIABLES DUE TO TIME CRUNCH ############

# for loop fail
# keep_feat_new = []
# for value in keep_feat:
#     for item in dummies:
#         keep_feat_new = [value for value in keep_feat if item not in value]
# keep_feat.size
keep_feat_new = ['GrLivArea',
                 'MSSubClass',
                 'LotArea',
                 'OverallQual',
                 'YearBuilt',
                 'YearRemodAdd',
                 'MasVnrArea',
                 'BsmtFinSF1',
                 'BsmtFinSF2',
                 'BsmtUnfSF',
                 'TotalBsmtSF',
                 '1stFlrSF',
                 '2ndFlrSF',
                 'LowQualFinSF',
                 'FullBath',
                 'BedroomAbvGr',
                 'TotRmsAbvGrd',
                 'Fireplaces',
                 'GarageYrBlt',
                 'GarageCars',
                 'GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MoSold','YrSold','ExterQual_Enc','BsmtExposure_Enc',
 'BsmtFinType1_Enc',
 'BsmtFinType2_Enc',
 'HeatingQC_Enc',
 'KitchenQual_Enc',
 'Functional_Enc',
 'FireplaceQu_Enc',
 'GarageFinish_Enc',
 'GarageQual_Enc',
 'PavedDrive_Enc',
 'PoolQC_Enc',
 'LandContour_HLS',
 'LotConfig_Inside']

In [13]:
X2 = X[keep_feat_new]
X2

,GrLivArea,MSSubClass,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,HeatingQC_Enc,KitchenQual_Enc,Functional_Enc,FireplaceQu_Enc,GarageFinish_Enc,GarageQual_Enc,PavedDrive_Enc,PoolQC_Enc,LandContour_HLS,LotConfig_Inside
2467,1363,20,14559,5,1951,2000,70.0,650.0,180.0,178.0,...,5,1,1,1,1,1,2,0,0,1
220,1969,60,11952,7,1977,1977,0.0,0.0,0.0,808.0,...,1,1,0,4,2,1,2,0,0,1
1101,1121,20,8546,4,2003,2004,0.0,0.0,0.0,1121.0,...,5,1,0,0,2,1,2,0,0,0
17,864,30,8248,3,1914,1950,0.0,41.0,0.0,823.0,...,1,1,0,0,0,0,0,0,0,1
58,2418,60,10542,7,1993,1994,651.0,1173.0,0.0,138.0,...,5,4,0,1,2,1,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,1510,70,9084,4,1940,1950,0.0,0.0,0.0,755.0,...,1,1,0,4,1,3,1,0,0,1
1731,984,45,6390,6,1954,1954,0.0,0.0,0.0,936.0,...,1,1,0,0,1,1,2,0,0,0
763,1584,90,8944,5,1967,1967,0.0,0.0,0.0,1584.0,...,1,1,3,0,1,1,2,0,0,1
835,1159,50,7506,5,1925,1950,0.0,0.0,0.0,747.0,...,1,1,0,0,1,3,0,0,0,0


In [14]:
### lasso round 2 with no dummy data
# clv = Lasso(random_state=0).fit(X2, y_train)
# search = GridSearchCV(Lasso(random_state=0, tol=.01), 
#                       {'alpha': np.arange(0.01,1000,1)},
#                      cv=5,
#         scoring='neg_mean_squared_error'
#                      )
# search.fit(X2, y_train)
# search.best_params_

# pickle.dump(search, open('flask_fsgs2.sav', 'wb'))
search = pickle.load(open('flask_fsgs2.sav', 'rb'))
search.best_params_

coefficients = search.best_estimator_.coef_
importance = np.abs(coefficients)
features = X2.columns
keep_feat = np.array(features)[importance > 0]

df_imp = pd.DataFrame(zip(keep_feat,importance)).sort_values(1, ascending = False)
df_imp = df_imp.loc[df_imp[1]>1000]
df_imp

,0,1
40,PoolQC_Enc,19105.999363
41,LandContour_HLS,13974.887280
3,OverallQual,12325.205211
15,BedroomAbvGr,7413.141515
35,Functional_Enc,4568.219127
14,FullBath,4018.120966
30,BsmtExposure_Enc,3378.295066
19,GarageCars,3273.159800
17,Fireplaces,2931.380102
29,ExterQual_Enc,2894.388674


In [20]:
######## 16 feature test --> 76.754% best accuracy  #############

# X3_train = X_train[df_imp[0]]
# X3_test = X_test[df_imp[0]]

In [33]:
########## KEEP 47 FEATURES LEFT BY LASSO ############

X3_train = X_train[keep_feat]
X3_test = X_test[keep_feat]

### Train Model

In [34]:
las2 = Lasso(random_state=0)
las2.fit(X3_train, y_train)

C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 609722361952.6511, tolerance: 1125382511.83553
  model = cd_fast.enet_coordinate_descent(


Lasso(random_state=0)

In [35]:
y_pred = las2.predict(X3_test)
r2_score(y_test, y_pred)

0.8964174730202954

In [24]:
######## SCALER ##########
# mms = MinMaxScaler()
# mms.fit(X3_train)
# X3_train = mms.transform(X3_train)
# pickle.dump(mms, open('MinMaxScaler.pkl', 'wb'))

In [25]:
# pipe = make_pipeline(
#             MinMaxScaler(),
#             Lasso()
# )
# pipe.fit(X3_train,y_train)
# print('pipe score',pipe.score(X3_test,y_test))     

pipe score -555.1643698365109


In [27]:
######### GRID SEARCH CV ##########

# Grid Search Cross Validation
# params = {'alpha': np.arange(-1000,1000,10),
#           'selection': ['cyclic','random']}
# clf = las.fit(X3_train, y_train)
# gs = GridSearchCV(clf, param_grid=params, cv=3)
# gs.fit(X3_train,y_train)
# print('best CV score:', gs.best_score_)
# print('best hyperparameter combination:', gs.best_params_)

C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2677970292080.2686, tolerance: 803720041.8887303
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3302171783075.2607, tolerance: 738675740.866675
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1871057384378.3374, tolerance: 707947171.4623001
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarn

C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3316075725665.4277, tolerance: 738675740.866675
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1920447343216.4365, tolerance: 707947171.4623001
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2113178161097.1523, tolerance: 803720041.8887303
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarn

C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3332760456773.5337, tolerance: 738675740.866675
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1979715293822.1924, tolerance: 707947171.4623001
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2039412090833.0134, tolerance: 803720041.8887303
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarn

C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3052160286967.9062, tolerance: 738675740.866675
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2038983244427.9165, tolerance: 707947171.4623001
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2092256185292.6567, tolerance: 803720041.8887303
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarn

C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3083972958528.7246, tolerance: 738675740.866675
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2088373203265.9497, tolerance: 707947171.4623001
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2176644711763.623, tolerance: 803720041.8887303
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarni

...

C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2607311055829.6045, tolerance: 707947171.4623001
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3268994495370.2466, tolerance: 803720041.8887303
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3387811681697.0693, tolerance: 738675740.866675
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarn

C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3343698238483.5493, tolerance: 803720041.8887303
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3451957999423.8877, tolerance: 738675740.866675
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2704059327102.8145, tolerance: 707947171.4623001
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarn

C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3436932826947.3433, tolerance: 803720041.8887303
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3532140896582.3594, tolerance: 738675740.866675
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2800807598376.0107, tolerance: 707947171.4623001
  model = cd_fast.enet_coordinate_descent(
C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarn

best CV score: 0.7657227203995006
best hyperparameter combination: {'alpha': -30, 'selection': 'cyclic'}


C:\Users\hzeig\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4934429616443.072, tolerance: 1125382511.83553
  model = cd_fast.enet_coordinate_descent(


In [28]:
# y_pred = gs.best_estimator_.predict(X3_test)
# r2_score(y_test, y_pred)

-563.7321474824735

### Pickle for Flask

In [36]:
# no grid search / no scaler lasso performed the best 
# --> not enough time to keep experimenting
model = las2
pickle.dump(model, open('dash_model.pkl','wb'))

### Dashboard Pipeline
- ~~dummify~~
- ~~scale~~
- predict

In [37]:
model = pickle.load(open('dash_model.pkl','rb'))
sample_input = X3_test.sample(1, random_state=0)
prediction = model.predict(sample_input)

In [38]:
sample_input

,GrLivArea,MSSubClass,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,HeatingQC_Enc,KitchenQual_Enc,Functional_Enc,FireplaceQu_Enc,GarageFinish_Enc,GarageQual_Enc,PavedDrive_Enc,PoolQC_Enc,LandContour_HLS,LotConfig_Inside
1944,984,20,9750,5,1959,1959,164.0,200.0,0.0,784.0,...,4,3,0,0,1,1,0,0,0,1


In [39]:
# calculate mean absolute error to provide prediction range in app 
y_pred = model.predict(X3_test)
mae = mean_absolute_error(y_test, y_pred)
mae

17454.143577839826

In [ ]:
#### for translating dummified data input.... not enough time

# sample_raw = {}

# index_dict = dict(zip(df.columns,range(df.shape[1])))

# new_vector = np.zeroes(297)
# try:
#     new_vector[index_dict[origin]] = 1
# except:
#     pass
# try:
#     new_vector[index_dict[destination]] = 1
# except:
#     pass
# try:
#     new_vector[index_dict[carrier]] = 1
# except:
#     pass